# Imports


In [65]:
import numpy as np
import pandas as pd

import snsql
from sklearn import metrics

# Importing the dataset


In [2]:
nov_2019 = pd.read_csv("../data/broadband_data_2019November.csv")
oct_2020 = pd.read_csv("../data/broadband_data_2020October.csv")
zipcode = pd.read_csv("../data/broadband_data_zipcode.csv")

In [15]:
nov_2019.head()

,ST,COUNTY ID,COUNTY NAME,BROADBAND AVAILABILITY PER FCC,BROADBAND USAGE
0,AL,1001,Autauga County,0.81,0.28
1,AL,1003,Baldwin County,0.88,0.30
2,AL,1005,Barbour County,0.59,0.18
3,AL,1007,Bibb County,0.29,0.07
4,AL,1009,Blount County,0.69,0.09


In [4]:
oct_2020.head()

,ST,COUNTY ID,COUNTY NAME,BROADBAND AVAILABILITY PER FCC,BROADBAND USAGE
0,AL,1001,Autauga County,0.8057,0.391
1,AL,1003,Baldwin County,0.8362,0.452
2,AL,1005,Barbour County,0.6891,0.324
3,AL,1007,Bibb County,0.3368,0.136
4,AL,1009,Blount County,0.758,0.199


## Basic EDA on these datasets


First let us clean the column names


In [3]:
nov_2019.columns = nov_2019.columns.str.strip().str.replace(" ", "_")
oct_2020.columns = oct_2020.columns.str.strip().str.replace(" ", "_")

In [17]:
nov_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143 entries, 0 to 3142
Data columns (total 5 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   ST                              3143 non-null   object
 1   COUNTY_ID                       3143 non-null   int64 
 2   COUNTY_NAME                     3143 non-null   object
 3   BROADBAND_AVAILABILITY_PER_FCC  3143 non-null   object
 4   BROADBAND_USAGE                 3143 non-null   object
dtypes: int64(1), object(4)
memory usage: 122.9+ KB


In [18]:
oct_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 5 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   ST                              3142 non-null   object
 1   COUNTY_ID                       3142 non-null   int64 
 2   COUNTY_NAME                     3142 non-null   object
 3   BROADBAND_AVAILABILITY_PER_FCC  3142 non-null   object
 4   BROADBAND_USAGE                 3142 non-null   object
dtypes: int64(1), object(4)
memory usage: 122.9+ KB


One interesting thing here to note is that both the `BROADBAND_AVAILABILITY_PER_FCC` and the `BROADBAND_USAGE` column are of object data type, when ideally they should have been float. This means that there are some anomalous data present in there.


In [6]:
nov_2019["BROADBAND_AVAILABILITY_PER_FCC"].unique()

array([' 0.81 ', ' 0.88 ', ' 0.59 ', ' 0.29 ', ' 0.69 ', ' 0.06 ',
       ' 0.78 ', ' 0.93 ', ' 0.82 ', ' 0.99 ', ' 0.66 ', ' 0.23 ',
       ' 0.63 ', ' 0.40 ', ' 0.13 ', ' 0.90 ', ' 0.80 ', ' 0.76 ',
       ' 0.84 ', ' 0.75 ', ' 0.77 ', ' 0.98 ', ' 0.92 ', ' 0.74 ',
       ' 0.96 ', ' 0.38 ', ' 0.64 ', ' 0.54 ', ' 0.00 ', ' 0.87 ',
       ' 0.83 ', ' 0.34 ', ' 0.79 ', ' 0.55 ', ' 0.91 ', ' 0.61 ',
       ' 0.48 ', ' 0.95 ', ' 0.60 ', ' -   ', ' 0.27 ', ' 0.50 ',
       ' 0.89 ', ' 0.44 ', ' 0.30 ', ' 0.42 ', ' 0.49 ', ' 0.37 ',
       ' 0.25 ', ' 0.85 ', ' 0.62 ', ' 0.71 ', ' 0.68 ', ' 0.47 ',
       ' 0.10 ', ' 0.52 ', ' 0.07 ', ' 1.00 ', ' 0.53 ', ' 0.45 ',
       ' 0.86 ', ' 0.57 ', ' 0.70 ', ' 0.21 ', ' 0.97 ', ' 0.43 ',
       ' 0.73 ', ' 0.56 ', ' 0.17 ', ' 0.14 ', ' 0.46 ', ' 0.58 ',
       ' 0.22 ', ' 0.72 ', ' 0.01 ', ' 0.36 ', ' 0.32 ', ' 0.94 ',
       ' 0.41 ', ' 0.39 ', ' 0.09 ', ' 0.18 ', ' 0.12 ', ' 0.35 ',
       ' 0.67 ', ' 0.28 ', ' 0.11 ', ' 0.19 ', ' 0.51 ', ' 0.20

There is a `' -   '` in there.


In [7]:
nov_2019[nov_2019["BROADBAND_AVAILABILITY_PER_FCC"] == " -   "]

,ST,COUNTY_ID,COUNTY_NAME,BROADBAND_AVAILABILITY_PER_FCC,BROADBAND_USAGE
52,AL,1105,Perry County,-,0.07
67,AK,2013,Aleutians East Borough,-,0.01
68,AK,2016,Aleutians West Census Area,-,0.02
70,AK,2050,Bethel Census Area,-,0.03
71,AK,2060,Bristol Bay Borough,-,0.04
73,AK,2070,Dillingham Census Area,-,0.07
81,AK,2164,Lake and Peninsula Borough,-,0.02
83,AK,2180,Nome Census Area,-,0.03
84,AK,2185,North Slope Borough,-,0.08
85,AK,2188,Northwest Arctic Borough,-,0.01


This means that these are missing values, and we will need to do some sort of imputation before making any differentially private data release.

However in this case we should leave them to be None.
Reason:

-   The broadband usage data depends on multiple factors that vary from state to state and county to county. Any imputation strategy would be like - impute with mean or median, but that takes into account global statistics, and would not yield correct results.

-   Moreover, it also does not fall within the scope of this project as the only goal here is to make a differentially private data release.


In [4]:
def deal_with_missing_data(
    dataset: pd.DataFrame,
    columns: list = ["BROADBAND_AVAILABILITY_PER_FCC", "BROADBAND_USAGE"],
) -> pd.DataFrame:
    """
    Utility function to clean the given columns of the dataframes.
    The anomalous values are replaced with None and the remaining values are appropriately preprocessed and converted to float.
    """
    dataset = dataset.copy()
    for c in columns:
        dataset[c] = dataset[c].replace(" -   ", None).str.strip().astype(float)

    return dataset

In [5]:
nov_2019 = deal_with_missing_data(nov_2019)
oct_2020 = deal_with_missing_data(oct_2020)

In [32]:
nov_2019.shape

(3143, 5)

# Differentially private data aggregations


In [75]:
def differential_privacy(
    dataset: pd.DataFrame,
    agg_column: str,
    epsilon: float,
    metadata: str,
    table_name: str,
) -> pd.DataFrame:
    """
    Utility function to implement differential privacy when performing a data aggregation. In this case, we want to find out the average broadband usage/average broadband coverage per state.

    Parameters:
    ----------
    dataset:    pandas dataframe. The broadband usage dataset

    agg_column: string. The column to aggregrate on. Should be one of ["BROADBAND_USAGE", "BROADBAND_COVERAGE_PER_FCC"]

    epsilon:    float. privacy parameter.

    delta:      float. privacy parameter.

    table_name: string. Name of the table. One of ["November2019", "October2020"]
    """
    privacy = snsql.Privacy(epsilon=epsilon, delta=0.1)
    connection = snsql.from_df(dataset, privacy=privacy, metadata=metadata)
    result = connection.execute_df(
        f"SELECT ST, AVG({agg_column}) AS {agg_column} FROM Broadband.{table_name} GROUP BY ST"
    )

    final = pd.merge(
        dataset.groupby("ST")[f"{agg_column}"].mean().reset_index(),
        result,
        on="ST",
        suffixes=("_ORIGINAL", "_DP"),
    )

    final["ABSOLUTE_ERROR"] = np.abs(
        final[f"{agg_column}_ORIGINAL"] - final[f"{agg_column}_DP"]
    )
    mae = metrics.mean_absolute_error(
        final[f"{agg_column}_ORIGINAL"], final[f"{agg_column}_DP"]
    )
    rmse = np.sqrt(
        metrics.mean_squared_error(
            final[f"{agg_column}_ORIGINAL"], final[f"{agg_column}_DP"]
        )
    )

    return final, mae, rmse

## November 2019 data


In [70]:
final, mae, rmse = differential_privacy(
    nov_2019.copy(),
    "BROADBAND_USAGE",
    0.1,
    "../data/broadband_data_2019November_metadata.yaml",
    "November2019",
)

In [71]:
final

,ST,BROADBAND_USAGE_ORIGINAL,BROADBAND_USAGE_DP,ABSOLUTE_ERROR
0,AK,0.178214,-0.145548,0.323763
1,AL,0.198955,0.137802,0.061153
2,AR,0.138133,0.117833,0.020300
3,CA,0.421897,0.690528,0.268631
4,CO,0.340938,0.680376,0.339439
5,FL,0.382836,0.266644,0.116192
6,GA,0.255094,0.200111,0.054984
7,IA,0.260303,0.200897,0.059406
8,ID,0.211591,0.417339,0.205748
9,IL,0.270784,0.191440,0.079344


In [72]:
print(f"Mean Absolute error: {mae:.4f}\nRoot Mean Squared Error: {rmse:.4f}")

Mean Absolute error: 0.2306
Root Mean Squared Error: 0.4059


## October 2020 data


In [76]:
final, mae, rmse = differential_privacy(
    oct_2020.copy(),
    "BROADBAND_USAGE",
    0.1,
    "../data/broadband_data_2020October_metadata.yaml",
    "October2020",
)

In [77]:
final

,ST,BROADBAND_USAGE_ORIGINAL,BROADBAND_USAGE_DP,ABSOLUTE_ERROR
0,AK,0.272172,0.036487,0.235685
1,AL,0.289597,0.283916,0.005681
2,AR,0.226627,-0.119705,0.346332
3,AZ,0.458333,0.000000,0.458333
4,CA,0.529052,0.350134,0.178917
5,CO,0.449375,0.270103,0.179272
6,FL,0.507284,0.538626,0.031342
7,GA,0.376956,0.410121,0.033165
8,HI,0.551800,-5.708750,6.260550
9,IA,0.403576,0.097200,0.306375


In [78]:
print(f"Mean Absolute error: {mae:.4f}\nRoot Mean Squared Error: {rmse:.4f}")

Mean Absolute error: 0.3684
Root Mean Squared Error: 1.0390
